In [52]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import imageio.v2 as imageio

# Данные матрицы смежности
adj_matrix = pd.read_csv('data/los_adj.csv', header=None)
speed_matrix = pd.read_csv('data/los_speed.csv')
adj_matrix = adj_matrix - np.eye(207)

# Создание графа
G = nx.Graph()

# Добавление рёбер без самопетлей
for i in range(len(adj_matrix)):
    for j in range(i+1, len(adj_matrix)):  # Исключаем самопетли и дублирование рёбер
        weight = adj_matrix.iloc[i, j]
        if weight != 0:  # Если вес не равен 0, добавляем ребро
            G.add_edge(i, j, weight=weight)

# Если planar_layout не сработает (граф не планарный), используем kamada_kawai_layout
pos = nx.kamada_kawai_layout(G)

# Функция для получения цвета
def get_color(value):
    return plt.cm.viridis(value)

# Находим максимальное значение скорости
max_speed = speed_matrix.max().max()

# Создание цветовой шкалы
sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(vmin=0, vmax=max_speed))
sm.set_array([])

# Создание кадров для GIF
filenames = []
for t in range(60, 132+1):  # По времени (строки матрицы)
    fig, ax = plt.subplots(figsize=(16, 10))
    часы = (t * 5) // 60
    минуты = (t * 5) % 60
    ax.set_title(f'Время: {часы:02d}:{минуты:02d}')
    node_colors = [get_color(speed_matrix.iloc[t, i] / max_speed) for i in G.nodes()]  # Нормализуем значения для цвета
    
    nx.draw(G, pos, ax=ax, with_labels=False, node_color=node_colors, node_size=150, font_size=6, 
        edge_color='gray', width=0.5, alpha=0.7)

    # Добавление цветовой шкалы
    cbar = fig.colorbar(sm, ax=ax)
    cbar.set_label('Скорость (км/час)')
    
    filename = f'data/img/frame_{t}.png'
    plt.savefig(filename)  # Сохраняем каждый кадр
    filenames.append(filename)
    plt.close(fig)

# Создание GIF
with imageio.get_writer('graph_animation.gif', mode='I', duration=0.1) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Очистка временных файлов
import os
for filename in filenames:
    os.remove(filename)

print("GIF создан!")

C:\Users\romab\AppData\Local\Temp\ipykernel_16656\886881748.py:60: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF создан!
